# Chapter 5 - Ensmble Methods

In [1]:
import sys
sys.path.append("../")
from utils import *

np.random.seed(7)

## Bias-Variance Trade-off

$\newcommand{\coloneqq}{\mathrel{\vcenter{:}}=}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\y}{\mathbf{y}}$

Let us compute the bias-variance trade-off graph for a problem of polynomial fitting. Recall, that the error decomposition for the MSE loss function is: $$ MSE_{\y}\left(\widehat{\y}\right)=\E\left[\left(\widehat{\y}-\y^*\right)^2\right] = Var\left(\widehat{\y}\right) + Bias^2\left(\widehat{\y}\right) $$

Where the bias and variances of estimators are defined as: $$ Bias\left(\widehat{\y}\right) \coloneqq \E\left[\widehat{\y}\right] - \y, \quad Var\left(\widehat{\y}\right)\coloneqq \E\left[\left(\widehat{\y}-\E\left[\widehat{\y}\right]\right)^2\right]$$

As the $\E\left[\widehat{\y}\right]$ is over the selection of the training sets, we will first defined the "ground truth" model and retrieve a set $\mathbf{X},\y$ from it. Then, we will repeatedly sample Gaussian noise $\varepsilon$ and fit a polynomial model over $\mathbf{X},\y+\varepsilon$. In the code below `y_` denotes the true $\y$ values and `y` the responses after adding the noise.

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

# Generate data according to a polynomial model of degree 4
model = lambda x: x**4 - 2*x**3 - .5*x**2 + 1
X = np.linspace(-1.6, 2, 60)
y = model(X).astype(np.float64)
X_train, X_test, y_train_, y_test_ = train_test_split(X, y, test_size=.5, random_state=13)


# The following functions recieve two matrices of the true values and the predictions
# where rows represent different runs and columns the different responses in the run
def variance(y_pred):
    return np.mean(np.var(y_pred - np.mean(y_pred, axis=0), axis=0, ddof=1))

def bias(y_pred, y_true):
    mean_y = y_pred.mean(axis=0)
    return np.mean((mean_y - y_true)**2)

def error(y_pred, y):
    return np.mean((y_pred - y)**2)



ks, repetitions = list(range(11)), 100
biases, variances, errors = np.zeros(len(ks)), np.zeros(len(ks)), np.zeros(len(ks))
for i, k in enumerate(ks):
    # Add noise to train and test samples
    y_train = y_train_[np.newaxis, :] + np.random.normal(0, 3, size=(repetitions, len(y_train_)))
    y_test  = y_test_ + np.random.normal(size=len(y_test_))
    
    # Fit model multiple times (each time over a slightly different training sample) and predict over test set
    y_preds = np.array([make_pipeline(PolynomialFeatures(k), LinearRegression())\
                            .fit(X_train.reshape(-1,1), y_train[j,:])\
                            .predict(X_test.reshape(-1,1))
                        for j in range(repetitions)])
    
    biases[i], variances[i], errors[i] = bias(y_preds, y_test_), variance(y_preds), error(y_preds, y_test_)


fig = go.Figure([
            go.Scatter(x=ks, y=biases, name=r"$Bias^2$"),
            go.Scatter(x=ks, y=variances, name=r"$Variance$"),
            go.Scatter(x=ks, y=biases+variances, name=r"$Bias^2+Variance$"),
            go.Scatter(x=ks, y=errors, name=r"$Generalization\,\,Error$")], 
        layout=go.Layout(title=r"$\text{Generalization Error Decomposition - Bias-Variance of Polynomial Fitting}$",
                         xaxis=dict(title=r"$\text{Degree of Fitted Polymonial}$"),
                         width=800, height=500))
fig.write_image(f"../figures/bias_variance_poly.png")
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: '../figures/bias_variance_poly.png'

## Committee Decisions

Let $X_1,\ldots,X_T\overset{iid}{\sim}Ber\left(p\right)$ taking values in $\left\{\pm1\right\}$, with the probability of each being correct being $p>0.5$. We can bound the probability of the committee being correct by: $$\mathbb{P}\left(\sum X_i > 0\right) \geq 1-\exp\left(-\frac{T}{2p}\left(p-\frac{1}{2}\right)^2\right)$$

Let us show this bounding below empirically by sampling increasing amount of such Bernoulli random variables, and to do so for different values of $p$.

In [ ]:
bound = np.vectorize(lambda p, T: 1-np.exp(-(T/(2*p))*(p-.5)**2))

ps  = np.concatenate([[.5001], np.linspace(.55, 1, 14)])
Ts = [1,5,10,15,20,25,50,75,100,125,150,175,200,250,300,400,500,600]

frames = []
for p in ps:
    theoretical = bound(p,Ts)
    empirical = np.array([[np.sum(np.random.choice([1, -1], T, p=[p, 1-p])) > 0 for _ in range(100)] for T in Ts])
    
    frames.append(go.Frame(data=[go.Scatter(x=Ts, y=theoretical, mode="markers+lines", name="Theoretical Bound",
                                            line=dict(color="grey", dash='dash')),
                                 go.Scatter(x=Ts, y=empirical.mean(axis=1), 
                                            error_y = dict(type="data", array=empirical.var(axis=1)),
                                            mode="markers+lines", marker_color="black", name="Empirical Probability")],
                           layout=go.Layout(
                               title_text=r"$\text{{Committee Correctness Probability As Function of }}\
                               T\text{{: }}p={0}$".format(round(p,3)),
                               xaxis=dict(title=r"$T \text{ - Committee Size}$"),
                               yaxis=dict(title=r"$\text{Probability of Being Correct}$", range=[0.0001,1.01]))))


fig = go.Figure(data=frames[0]["data"],
        frames=frames[1:], 
        layout=go.Layout(
            title=frames[0]["layout"]["title"],
            xaxis=frames[0]["layout"]["xaxis"],
            yaxis=frames[0]["layout"]["yaxis"],
            updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(frame_duration=1000), 
                                                       AnimationButtons.pause()])] ))

animation_to_gif(fig, "../figures/committee_decision_correctness.gif", 700, width=600, height=450)
fig.show()

In this case, of uncorrelated committee members, we have shown the variance in the committee decision is: $$ Var\left(\sum X_i\right) = \frac{4}{T}p\left(1-p\right)$$
Let us simulate such a scenario and see what is the empirical variance we achieve

In [ ]:
ps  = np.concatenate([[.5001], np.linspace(.55, 1, 10)])
Ts = [1,5,10,15,20,25,50,75,100,125,150,175,200,250,300,400,500,600]

    results = np.array([np.var(np.random.binomial(Ts, p, (10000, len(Ts))) >= (np.array(Ts)/2), axis=0, ddof=1) for p in ps])

df = pd.DataFrame(results, columns=Ts, index=ps)
fig = go.Figure(go.Heatmap(x=df.columns.tolist(), y=df.index.tolist(), z=df.values.tolist(), colorscale="amp"),
          layout=go.Layout(title=r"$\text{Variance of Committee Decision - Independent Members}$", 
                           xaxis=dict(title=r"$T\text{ - Committee Size}$", type="category"),
                           yaxis=dict(title=r"$p\text{ - Member Correctness Probability}$"),
                           width=800, height=500))

fig.write_image("../figures/uncorrelated_committee_decision.png")
fig.show()

For a set of correlated random variables, with correlation coefficient of $\rho$ and variance of $\sigma^2$, the variane of the committee's decision is: $$ Var\left(\sum X_i\right) = \rho \sigma^2 + \frac{1}{T}\left(1-\rho\right)\sigma^2 $$
Let us set $\sigma^2$ and investigate the relation between $\rho$ and $T$.

In [ ]:
sigma = round((lambda p: p*(1-p))(.6), 3)
repeats = 10000
rho = np.linspace(0,1, 10)
Ts = np.array([1,5,10,15,20,25,50,75,100,125,150,175,200,250,300,400,500,600])

variances = np.zeros((len(rho), len(Ts)))
for i, r in enumerate(rho):
    # Perform `repetitions` times T Bernoulli experiments
    decisions = np.random.binomial(1, sigma, size=(repeats, max(Ts)))
    change = np.c_[np.zeros(decisions.shape[0]), np.random.uniform(size=(repeats, max(Ts)-1)) <= r]
    correlated_decisions = np.ma.array(decisions, mask=change).filled(fill_value=decisions[:,0][:, None])
    correlated_decisions[correlated_decisions == 0] = -1

    variances[i,:] = np.var(np.cumsum(correlated_decisions, axis=1) >= 0, axis=0)[Ts-1]
    
df = pd.DataFrame(variances, columns=Ts, index=rho)
fig = go.Figure(go.Heatmap(x=df.columns.tolist(), y=df.index.tolist(), z=df.values.tolist(), colorscale="amp"),
          layout=go.Layout(title=rf"$\text{{Variance of Committee Decision - Correlated Committee Members - Member Decision Variance }}\sigma^2 = {sigma}$", 
                           xaxis=dict(title=r"$T\text{ - Committee Size}$", type="category"),
                           yaxis=dict(title=r"$\rho\text{ - Correlation Between Members}$"),
                           width=500, height=300))

fig.write_image("../figures/correlated_committee_decision.png")
fig.show()

## Bootstrapping
### Empirical CDF

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import norm

data = np.random.normal(size=10000)
frames = []
for m in [5,10, 15, 20, 25, 50, 75, 100, 150, 200, 250, 500, 750, 1000,1500, 2000, 2500, 5000, 7500, 10000]:
    ecdf = ECDF(data[:m])
    frames.append(go.Frame(
        data = [
            go.Scatter(x=data[:m], y=[-.1]*m, mode="markers", marker=dict(size=5, color=norm.pdf(data[:m])), name="Samples"),
            go.Scatter(x=ecdf.x, y=ecdf.y, marker_color="black", name="Empirical CDF"),                
            go.Scatter(x=np.linspace(-3,3,100), y=norm.cdf(np.linspace(-3,3,100), 0, 1), mode="lines", 
                       line=dict(color="grey", dash='dash'), name="Theoretical CDF")],
        layout = go.Layout(title=rf"$\text{{Empirical CDF of }}m={m}\text{{ Samples Drawn From }}\mathcal{{N}}\left(0,1\right)$")
    ))

fig = go.Figure(data = frames[0].data, frames=frames[1:], 
                layout=go.Layout(title=frames[0].layout.title,
                                 updatemenus=[dict(type="buttons", buttons=[AnimationButtons.play(frame_duration=1000), 
                                                                            AnimationButtons.pause()])]))


animation_to_gif(fig, "../figures/empirical_cdf.gif", 700, width=600, height=450)
fig.show()

### Collection of Bagged Decision Trees

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
np.random.seed(7)

class TreeEnsemble:
    def __init__(self, num_of_trees, d):
        self.trees = []
        self.num_of_trees = num_of_trees
        self.depth = d
        
    def fit_bootstrap_tree(self, X, y):
        idx = resample(range(len(X)), replace = True, n_samples = len(X))
        return DecisionTreeClassifier(max_depth=self.depth).fit(X[idx], y[idx])
        
    def fit(self, X, y):
        self.trees = [self.fit_bootstrap_tree(X, y) for _ in range(self.num_of_trees)]
        return self
    
    def predict(self, X):
        return self.partial_predict(X)
    
    def partial_predict(self, X):
        samples_proba = self.predict_proba(X)  # samples_proba[0]: probablity for class "0". samples_proba[1]: probablity for class "1".
        return np.argmax(samples_proba, axis=1)  # returns for each sample, the index of the maximum probability class
        
    def predict_proba(self, X):
        # This function calculates the probability of each class ("0"/"1") for each sample.
        trees_mean = np.mean([self.trees[t].predict(X) for t in range(self.num_of_trees)], axis=0)
        return np.stack((1-trees_mean, trees_mean), axis=1)

In [6]:
# Generate 2D data to of 2 classes
d, n_train, n_test = 5, 2000, 500
X, y = create_data_bagging_utils(d=d, n_samples = n_train + n_test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=n_test, random_state=42)
depth = 2  # The depth of the single decision tree and of each tree in the ensemble

tree = DecisionTreeClassifier(max_depth=depth).fit(X_train, y_train)
tree_loss = np.sum(tree.predict(X_test) != y_test) / len(y_test)

ensemble_losses = []
nums = list(range(1, 351))
for num_trees in nums:
    ensemble = TreeEnsemble(num_trees, depth).fit(X_train, y_train)
    ensemble_losses.append(np.sum(ensemble.predict(X_test) != y_test) / len(y_test))

go.Figure([go.Scatter(x=nums, y=ensemble_losses, mode='lines', name=r"$\text{Ensemble}$"), 
                 go.Scatter(x=nums, y=[tree_loss for i in nums], mode='lines', name=r"$\text{Single Tree}$")],
                layout=go.Layout(title=rf"$\text{{Loss as a Function of the Ensemble's size}}$", margin=dict(t=100),
                                 xaxis_title=r"$\text{Ensemble's size}$", yaxis_title=r"$\text{Loss}$",
                                 width=700, height=370))

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

class StagedAdaBoostClassifier(AdaBoostClassifier):
    def __init__(self, **kwargs):
        super().__init__(*kwargs)
        self.sample_weights = []

    def _boost(self, iboost, X, y, sample_weight, random_state):
        self.sample_weights.append(sample_weight.copy())
#         self.res_list.append(super()._boost(iboost, X, y, sample_weight, random_state))
#         return self.res_list[-1]
        return super()._boost(iboost, X, y, sample_weight, random_state)

    def _iteration_callback(self, iboost, X, y, sample_weight, 
                            estimator_weight = None, estimator_error = None):
        self.sample_weights.append(sample_weight.copy())
        
        
from sklearn.datasets import make_gaussian_quantiles

# Construct dataset of two sets of Gaussian quantiles
X1, y1 = make_gaussian_quantiles(cov=2., n_samples=50, n_features=2, n_classes=2, random_state=1)
X2, y2 = make_gaussian_quantiles(mean=(3, 3), cov=1.5, n_samples=50, n_features=2, n_classes=2, random_state=1)
X, y = np.concatenate((X1, X2)), np.concatenate((y1, - y2 + 1))


# Form grid of points to use for plotting decision boundaries 
lims = np.array([X.min(axis=0), X.max(axis=0)]).T + np.array([-.2, .2])
xx, yy = list(map(np.ravel, np.meshgrid(np.arange(*lims[0], .2), np.arange(*lims[1], .2))))


# Fit AdaBoost classifier over training set
model = StagedAdaBoostClassifier().fit(X, y)
# Retrieve model train error at each iteration of fitting
staged_scores = list(model.staged_score(X, y))
# Predict labels of grid points at each iteration of fitting
staged_predictions = np.array(list(model.staged_predict(np.vstack([xx, yy]).T)))

In [ ]:
# Create animation frames
frames = []
for i in range(len(staged_predictions)):
    frames.append(go.Frame(
        data=[
            # Scatter of sample weights
            go.Scatter(x=X[:,0], y= X[:,1], mode='markers', showlegend=False, marker=dict(color=y, colorscale=class_colors(2),
                       size=np.maximum(230*model.sample_weights[i]+1, np.ones(len(model.sample_weights[i]))*5)),
                       xaxis="x", yaxis="y"), 
            
            # Staged decision surface 
            go.Scatter(x=xx,  y=yy, marker=dict(symbol = "square", colorscale=custom, color=staged_predictions[i,:]), 
                       mode='markers', opacity = 0.4, showlegend=False, xaxis="x2", yaxis="y2"),
            
            # Scatter of train samples with true class
            go.Scatter(x=X[:,0],  y=X[:,1], mode='markers', showlegend=False, xaxis="x2", yaxis="y2",
                       marker=dict(color=y, colorscale=class_colors(2), symbol=class_symbols[y])),
            
            # Scatter of staged score
            go.Scatter(x=list(range(i)), y=staged_scores[:i], mode='lines+markers', showlegend=False, marker_color="black",
                       xaxis="x3", yaxis="y3")
        ],
        layout = go.Layout(title = rf"$\text{{AdaBoost Training - Iteration }}{i+1}/{len(staged_predictions)}$)"),
        traces=[0, 1, 2, 3]))    

    
fig = make_subplots(rows=2, cols=2, row_heights=[350, 200],
                    subplot_titles=(r"$\text{Sample Weights}$", r"$\text{Decisions Boundaries}$", 
                                    r"$\text{Ensemble Train Accuracy}$"),
                    specs=[[{}, {}], [{"colspan": 2}, None]])\
    .add_traces(data=frames[0].data, rows=[1,1,1,2], cols=[1,2,2,1])\
    .update(frames = frames)\
    .update_layout(title=frames[0].layout.title,
                   updatemenus = [dict(type="buttons", buttons=[AnimationButtons.play(), AnimationButtons.pause()])], 
                   width=600, height=550, margin=dict(t=100))\
    .update_yaxes(range=[min(staged_scores)-.1, 1.1], autorange=False, row=2, col=1)\
    .update_xaxes(range=[0, len(frames)], autorange=False, row=2, col=1)

animation_to_gif(fig, "../figures/adaboost.gif", 1000, width=600, height=550)
fig.show()